In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from omegaconf import OmegaConf

from typing import Tuple

from boozie.get_data import load_wine

In [16]:
cfg = OmegaConf.load("../boozie/conf/config.yaml")

In [17]:
df = load_wine()

In [18]:
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [19]:
df.quality.value_counts().sort_index()

quality
3      20
4     163
5    1457
6    2198
7     880
8     175
9       5
Name: count, dtype: int64

In [20]:
def transform_to_star_score(scores: pd.Series) -> pd.Series:
    result = pd.Series(index=scores.index)
    result[scores.isin(range(5))] = 1
    result[scores == 5] = 2
    result[scores == 6] = 3
    result[scores == 7] = 4
    result[scores > 7] = 5

    return result

transform_to_star_score(df.quality).value_counts()

3.0    2198
2.0    1457
4.0     880
1.0     183
5.0     180
Name: count, dtype: int64

In [21]:
df["stars"] = transform_to_star_score(df.quality)

In [22]:
def extract_samples(
    df: pd.DataFrame,*, score_name: str, samples: dict, random_state: int = 42
) -> Tuple[pd.DataFrame, pd.DataFrame]:
    
    df = df.copy()
    samples_df = pd.DataFrame()

    for name, score in samples.items():
        subset = df[df[score_name] == score]

        record = (subset.sample(random_state=random_state)
                        .assign(name=name))
        df = df.drop(record.index)
        samples_df = pd.concat([samples_df, record])

    return samples_df, df

In [23]:
samples_df, tdf = extract_samples(df, score_name="stars", samples=cfg.samples)

In [24]:
samples_df

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,stars,name
672,7.1,0.32,0.34,14.5,0.039,46.0,150.0,0.9950,3.38,0.50,12.5,8,5.0,Schmidt
1822,8.2,0.42,0.29,4.1,0.030,31.0,100.0,0.9911,3.00,0.32,12.8,7,4.0,Penfolds
1025,6.2,0.15,0.46,1.6,0.039,38.0,123.0,0.9930,3.38,0.51,9.7,6,3.0,Antinori
1672,6.6,0.26,0.31,4.8,0.138,41.0,168.0,0.9951,3.20,0.38,9.3,5,2.0,Vega Sicilia
445,7.1,0.32,0.32,11.0,0.038,16.0,66.0,0.9937,3.24,0.40,11.5,3,1.0,S-Budget


In [ ]:
tdf.loc[2420, :]

fixed acidity             8.10000
volatile acidity          0.40000
citric acid               0.32000
residual sugar            7.90000
chlorides                 0.03100
free sulfur dioxide      23.00000
total sulfur dioxide    118.00000
density                   0.99176
pH                        3.05000
sulphates                 0.46000
alcohol                  13.30000
quality                   7.00000
Name: 2420, dtype: float64